In [ ]:
%pylab inline
rcParams['figure.figsize'] = (10, 4) #wide graphs by default
from __future__ import print_function
from __future__ import division
from IPython.display import Audio

# Audio Filters

In [ ]:
signal = random.random(2048) * 2 - 1

In [ ]:
spectrum, _, _, _ = specgram(signal, interpolation='nearest', Fs=44100)
colorbar()
pass

In [ ]:
spectrum.shape

In [ ]:
type(spectrum)

How do we account for the numbers above? How do we confirm our guesses?

In [ ]:
2048 / 256

In [ ]:
plot(spectrum[:, 10])
pass

In [ ]:
filtered = (signal + r_[0, signal[:-1]]) # what's this doing?
spectrum2, _, _, _ = specgram(filtered, Fs=44100)
colorbar()
pass

In [ ]:
plot(spectrum2[:, 10])
pass

In [ ]:
plot(spectrum[:, 10], label='original')
plot(spectrum2[:, 10], label='filtered')
legend()
pass

In [ ]:
semilogx(spectrum[:, 10], label='original')
semilogx(spectrum2[:, 10] / 4.0, label='filtered')
legend()
pass

In [ ]:
plot(spectrum[:, 10], label='original')
plot(spectrum2[:, 10] / 4.0, label='filtered')
legend()
pass

Difference equation:

$$y(n) = b_0x(n) + b_1x(n-1) + ... + b_Mx(n-M)$$

for this simple filter:

$$y(n) = x(n) + x(n-1)$$

$$y(n) = 0.25x(n) + 0.25x(n-1)$$

You can estimate the effect of frequency from the difference equation!

In [ ]:
from scipy.signal import freqz
frequency, response = freqz([1, 1])
semilogy(frequency, abs(response))
title('Frequency Response')
grid()

In [ ]:
plot(frequency, angle(response))
title('Phase Response')
grid()

In [ ]:
filtered2 = (1 * signal + 0.5 * r_[0, signal[:-1]])
spectrum3, _, _, _ = specgram(filtered2, Fs=44100)
pass

$$y(n) = x(n) + 0.5 \cdot x(n-1)$$

In [ ]:
plot(spectrum3[:, 10])
pass

In [ ]:
plot(spectrum[:, 10])
plot(spectrum2[:, 10] / 4.0)
plot(spectrum3[:, 10] / (1.5**2))
pass

In [ ]:
from scipy.signal import freqz
frequency, response = freqz([1, 1])
semilogy(frequency, abs(response), label='1 1')

frequency, response = freqz([1, 0.5])
semilogy(frequency, abs(response), label='1 0.5')
title('Frequency Response')
legend()
grid()

In [ ]:
f, r = freqz([1, 1])
semilogy(f, abs(r))

f, r = freqz([1, 2])
semilogy(f, abs(r))

f, r = freqz([1, 0.5])
semilogy(f, abs(r))

title('Frequency Response')
grid()

In [ ]:
f, r = freqz([1, -1])
semilogy(f, abs(r))
title('Frequency Response')
grid()

What is this? How is it different?

# Transfer function

$$ Y(z) = H(z) \cdot X(z) $$

$$ H(z) = \frac{Y(z)}{X(z)} $$


# z-transform

$$\mathcal{Z}[x(n-M)] = z^{-M}X(z)$$

z-transform on the difference equation:

$$Y(z) = b_0z^{0}X(z) + b_1z^{-1}X(z) + ... + b_Mz^{-M}X(z) - a_1z^{-1}Y(z) - a_2z^{-2}Y(z) - ... - a_Mz^{-M}Y(z)$$

$$ [1 + a_1z^{-1} + a_2z^{-2} - ... + a_Mz^{-M}] \cdot Y(z) = [b_0z^{0} + b_1z^{-1} + ... + b_Mz^{-M}]\cdot X(z) $$

$$H(z) = \frac{Y(z)}{X(z)} = \frac{b_0z^{0} + b_1z^{-1} + ... + b_Mz^{-M}}{ 1 + a_1z^{-1} - a_2z^{-2} - ... - a_Mz^{-M}}$$

Really, this is just using a different notation to represent what we have already been saying:

$$b_0x(n) + b_1x(n-1) + ... + b_Mx(n-M) \text{  becomes  } b_0z^{0} + b_1z^{-1} + ... + b_Mz^{-M}$$

$$\text{For } X(z) \text{:   } x(n) = z^{0}, x(n - 1) = z^{-1}, x(n - 2) = z^{-2}, \text{and so on...}$$

Probably the most intuitive way of thinking about the $z$ operator is as a _delay_. $z^{-1}$ is the signal _delayed_ by $1$ sample. Or, in the context of implementing filters in code, then $z^{0}$ means use the current sample and $z^{-1}$ means use the previous sample.


# One-pole filters

$$y(n) = b_0x(n) - a_1y(n-1)$$

In [ ]:
from scipy.signal import lfilter
filtered4 = lfilter([1], [1, 1], signal)
specgram(filtered4, Fs=44100)
pass

In [ ]:
f, Y = freqz([1], [1, 1])
semilogy(f, abs(Y))
grid()
pass

In [ ]:
f, Y = freqz([1], [1, 0.5])
semilogy(f, abs(Y))
grid()
pass

In [ ]:
f, Y = freqz([1], [1, -0.5])
semilogy(f, abs(Y))
pass

In [ ]:
f, Y = freqz([1], [1, -0.5])
plot(f, angle(Y))
ylabel('phase')
pass

$$H(z) = \frac{b_0z^{0} + b_1z^{-1} + \cdots + b_Mz^{-M}}{ a_0z^{0} + a_1z^{-1} + a_2z^{-2}  + \cdots + a_Mz^{-M}}$$

 * FIR filters are always stable
 * IIR filters can produce steeper filters with smaller orders

# Pole-zero analysis

In [ ]:
from scipy.signal import tf2zpk

tf2zpk([1, 1], [1])

In [ ]:
tf2zpk([1], [1, 1])

In [ ]:
tf2zpk([2, 2], [1])

In [ ]:
def PoleZeroPlot(b, a):
    (zeros,poles,gain) = tf2zpk(b, a)
    angle = np.linspace(-np.pi,np.pi,50)
    cirx = np.sin(angle)
    ciry = np.cos(angle)
    figure()
    plot(poles.real, poles.imag, 'x', zeros.real, zeros.imag, 'o', cirx,ciry, 'k-')
    grid()
    
    xlim((-2, 2))
    xlabel('Real')
    ylim((-1.5, 1.5))
    ylabel('Imag')
    gcf().set_figwidth(5)
    return (zeros,poles,gain)

In [ ]:
PoleZeroPlot([1, 1], [1])

In [ ]:
PoleZeroPlot([1], [1, 1])

In [ ]:
PoleZeroPlot([1], [1, 1.1])

When a pole is outside the unit circle in the z-plane the filter is unstable! What does unstable mean? See [What is the exact meaning of unstable system in DSP?](https://dsp.stackexchange.com/questions/8001/what-is-the-exact-meaning-of-unstable-system-in-dsp) or [Is an Unstable Filter Usable?](https://www.kvraudio.com/forum/viewtopic.php?t=179603). Basically, if you use an unstable filter, your filtered signal might "blow up", that is go to infinity or negative infinity or both by way of oscillation. In the context of digital signals, this may cause clipping, overflow, or NaNs. Your filter may take a long time to blow up and it might only blow up for certain input signals.

In [ ]:
PoleZeroPlot([1], [1.1, 1.1])

# Comb filters

$$y(n) = b_0x(n) + b_m\cdot x(n-m)$$

In [ ]:
b = [1, 0, 0, 0, 0, 0, 0, 1]
#       1  2  3  4  5  6  7
# m = 7
a = [1]
f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
b = [1, 0, 0, 0, 0, 0, 1]
a = [1]
f, r = freqz(b, a)
plot(f, abs(r))
grid()
pass

In [ ]:
b = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
a = [1]
f, r = freqz(b,a)
semilogy(f, abs(r))
grid()

twinx()

plot(f, angle(r), 'r')
pass

In [ ]:
b = [1, 0, 0, 0, 0, 0, 0, -1]
a = [1]
f, r = freqz(b, a)
plot(f, abs(r))

b = [1, 0, 0, 0, 0, 0, 0, 1]
a = [1]
f, r = freqz(b, a)
plot(f, abs(r))
pass

## IIR Comb-fiters

$$y(n) = b_0x(n) + b_m\cdot x(n-m) - a_m\cdot y(n-m)$$

In [ ]:
b = [1, 0, 0, 0, 0, 0, 0, -1]
a = [1, 0, 0, 0, 0, 0, 0, 1]
f, r = freqz(b, a)
semilogy(f, abs(r))
pass

In [ ]:
b = [1]
a = [1, 0, 0, 0, 0, 0, 0, 1]
f, r = freqz(b, a, worN=8192)
semilogy(f, abs(r))
pass

## [Two-zero filters](https://ccrma.stanford.edu/~jos/fp/Two_Zero.html)

In [ ]:
b = [1, 1, 1]
a = [1]
f, r = freqz(b, a)
semilogy(f, abs(r))
pass

$$\frac{b_1}{b_0} = -2R\cos(\Theta_c)$$
$$\frac{b_2}{b_0} = R^2$$

In [ ]:
Radius = 0.8
Angle = 1.0
b = [1, -2 * Radius * cos(Angle), Radius**2]
a = [1]

f, r = freqz(b, a)
semilogy(f, abs(r))
pass

In [ ]:
Radius = 1
Angle = 2.0
b = [1, -2 * Radius * cos(Angle), Radius**2]
a = [1]

f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
Radius = 1
Angle = 1.5
b = [1, -2 * Radius * cos(Angle), Radius**2]
a = [1]

f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
PoleZeroPlot(b, a)

## [Two-pole filters](https://ccrma.stanford.edu/~jos/fp/Two_Pole.html)

$$a_1 = -2R\cos(\theta_c)$$
$$a_2 = R^2$$

In [ ]:
Radius = 1
Angle = 2.0
b = [1]
a = [1, -2 * Radius * cos(Angle), Radius**2]

f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
poles, zeros, k = PoleZeroPlot(b, a)

In [ ]:
abs(zeros)

In [ ]:
Radius = 0.9
Angle = 1.5
b = [1]
a = [1, -2 * Radius * cos(Angle), Radius**2]

f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
poles, zeros, k = PoleZeroPlot(b, a)

## [Biquad Filter](https://en.wikipedia.org/wiki/Digital_biquad_filter)

So named for the two [quadratic_functions](https://en.wikipedia.org/wiki/Quadratic_function), one on the top and one on the bottom:

$$H(z) = \frac{b_0z^{0} + b_1z^{-1} +  b_2z^{-2}}{ a_0z^{0} + a_1z^{-1} + a_2z^{-2}}$$

These are used a lot. See [Cookbook formulae for audio EQ biquad filter coefficients](http://www.musicdsp.org/files/Audio-EQ-Cookbook.txt) for recipes.

In [ ]:
# low shelf-filter

Fs = 44100
f0 = 10000.0
dBgain = 30.0
S = 1.0 # shelf slope
# -----------------------
A  = 10**(dBgain/40)

w0 = 2*pi*f0/Fs
alpha = sin(w0)/2 * sqrt( (A + 1/A)*(1/S - 1) + 2 ) 
       
b0 =    A*( (A+1) - (A-1)*cos(w0) + 2*sqrt(A)*alpha )
b1 =  2*A*( (A-1) - (A+1)*cos(w0)                   )
b2 =    A*( (A+1) - (A-1)*cos(w0) - 2*sqrt(A)*alpha )
a0 =        (A+1) + (A-1)*cos(w0) + 2*sqrt(A)*alpha
a1 =   -2*( (A-1) + (A+1)*cos(w0)                   )
a2 =        (A+1) + (A-1)*cos(w0) - 2*sqrt(A)*alpha

In [ ]:
w, h = freqz([b0, b1, b2], [a0, a1, a2])
semilogy(w, abs(h))
pass

In [ ]:
Fs = 44100
f0 = 10000.0
dBgain = -20.0
S = 2.0 # shelf slope

A  = 10**(dBgain/40)

w0 = 2*pi*f0/Fs
alpha = sin(w0)/2 * sqrt( (A + 1/A)*(1/S - 1) + 2 ) 
    
    
b0 =    A*( (A+1) - (A-1)*cos(w0) + 2*sqrt(A)*alpha )
b1 =  2*A*( (A-1) - (A+1)*cos(w0)                   )
b2 =    A*( (A+1) - (A-1)*cos(w0) - 2*sqrt(A)*alpha )
a0 =        (A+1) + (A-1)*cos(w0) + 2*sqrt(A)*alpha
a1 =   -2*( (A-1) + (A+1)*cos(w0)                   )
a2 =        (A+1) + (A-1)*cos(w0) - 2*sqrt(A)*alpha

w, h = freqz([b0, b1, b2], [a0, a1, a2])
plot(w,abs(h))
pass

In [ ]:
shelved = lfilter([b0, b1, b2], [a0, a1, a2], signal)
specgram(shelved, Fs=44100)
pass

In [ ]:
PoleZeroPlot([b0, b1, b2], [a0, a1, a2])
pass

# Filter families

<img src="http://upload.wikimedia.org/wikipedia/commons/5/5c/Electronic_linear_filters.svg" width="100%">

## [Chebyshev Filters](http://en.wikipedia.org/wiki/Chebyshev_filter)

* Type I: Ripple in the pass-band only
* Type II: Ripple in the stop-band only

In [ ]:
from scipy.signal import cheby1

b, a = cheby1(4, 0.5, 0.1)
b, a

In [ ]:
w, h = freqz(b, a)
plot(w, np.abs(h), 'b')
pass

In [ ]:
PoleZeroPlot(b, a)

In [ ]:
ripples = [2, 0.5, 0.1, 0.01]

for ripple in ripples:
    b, a = cheby1(4, ripple, 0.6)
    f, r = freqz(b, a)
    plot(f, abs(r))

legend(ripples)
title('Different ripple values for a Chebyshev I filter')
grid()

In [ ]:
order = [2, 3, 4, 6]

for o in order:
    b, a = cheby1(o, 2, 0.6)
    f, r = freqz(b, a)
    plot(f, abs(r))


legend(order)
title('Different orders for a Chebyshev I filter')
grid()

In [ ]:
from scipy.signal import cheby2

ripple = [12, 15, 20, 40]

for r in ripples:
    b, a = cheby2(4, r, 0.6)
    f, r = freqz(b, a)
    plot(f, abs(r))

legend(ripple)
title('Different ripple values for a Chebyshev II filter')
grid()

## Butterworth, Elliptic and Bessel filters

In [ ]:
from scipy.signal import iirdesign

In [ ]:
Wp = 0.5  # Cutoff frequency 
Ws = 0.6   # Stop frequency 
Rp = 0.1     # passband maximum loss (gpass)
As = 60      # stoppand min attenuation (gstop)
b, a = iirdesign(Wp, Ws, Rp, As, ftype='butter')
f, r = freqz(b,a)
plot(f, abs(r))
twinx()
plot(f, angle(r), 'r')

title('Butterworth filter')
grid()

In [ ]:
Wp = 0.5  # Cutoff frequency 
Ws = 0.6   # Stop frequency 
Rp = 1     # passband maximum loss (gpass)
As = 20      # stoppand min attenuation (gstop)

types = ['butter', 'ellip', 'cheby1', 'cheby2']

for t in types:
    b, a = iirdesign(Wp, Ws, Rp, As, ftype=t)
    f, r = freqz(b, a)
    plot(f, abs(r))
    
legend(types)
title('Filters')
grid()

You can use the *iirfilter* function from scipy.signal to design and filter in a single step.

http://docs.scipy.org/doc/scipy/reference/signal.html#filter-design

## Approximating arbitrary responses

There are several techniques that allow you to arbitrarily define target filter responses.

In [ ]:
from scipy.signal import firwin2

freqs = [0.0, 0.5, 1.0]
gains = [1.0, 1.0, 0.0]
order = 150

taps = firwin2(order, freqs, gains)

f, r = freqz(taps)
plot(f, abs(r))
twinx()
plot(f, angle(r), 'r')
pass

In [ ]:
freqs = [0.0, 0.3, 0.5, 0.8, 1.0]
gains = [1.0, 0.2, 0.5, 0.1, 0.0]
order = 200

taps = firwin2(order, freqs, gains)

f, r = freqz(taps)
plot(f, abs(r))
twinx()
plot(f, angle(r), 'r')
pass

In [ ]:
shaped = lfilter(taps, [1], signal)
specgram(shaped, Fs=44100)
pass

## [Remez filter design](https://en.wikipedia.org/wiki/Remez_algorithm)
[Parks-McClellan Filter design](https://en.wikipedia.org/wiki/Parks%E2%80%93McClellan_filter_design_algorithm) is a popular refinement of Remez specifically for FIR filters.

In [ ]:
from scipy.signal import remez
        
freqs = [0, 0.1, 0.2, 0.4, 0.45, 0.5]
gains = [0, 1, 0]

taps = remez(50, freqs, gains, type='bandpass')

f, r = freqz(taps)
plot(f/(2*pi), abs(r))
twinx()
plot(f/(2*pi), angle(r), 'r')
pass

In [ ]:
from scipy.signal import remez
        
freqs = [0, 0.1, 0.3, 0.4, 0.45, 0.5]
gains = [0, 1, 0]

taps = remez(16, freqs, gains, type='bandpass')

f, r = freqz(taps)
plot(f/(2*pi), abs(r))
twinx()
plot(f/(2*pi), angle(r), 'r')

## [All-pass filters](https://en.wikipedia.org/wiki/All-pass_filter)

> a signal processing filter that passes all frequencies equally in gain, but changes the phase relationship among various frequencies. It does this by varying its phase shift as a function of frequency.

[What are allpass filters and what are they used for?](http://www.uaudio.com/blog/allpass-filters)

> ...neither emphasize nor de-emphasize any part of the spectrum. Rather, they displace signals in time as a function of frequency. The time displacement accomplished by an allpass filter is specified by its phase response. ... If impulse response lengths are kept low, these filters can modify signal phase in a transparent way. With longer impulse responses, allpass filters can be used to create audible effects while preserving the spectral balance of a signal.


There are many ways of implmenting an all-pass filter. One is:

$$y(n) = -gx(n) + x(n-D) + gy(y-D)$$

In [ ]:
from scipy.signal import freqz
g = 0.9 # g is for gain?
b = [-g, 0, 0,  1]
a = [ 1, 0, 0, -g]

# what is the value of D here?

f, r = freqz(b, a)
plot(f, abs(r))
pass

In [ ]:
plot(f, angle(r))
xlabel('freq')
ylabel('angle')
pass

In [ ]:
signal = sin(linspace(0, 2*pi*5, 1000, endpoint=False)) + sin(linspace(0, 2*pi*30, 1000, endpoint=False))
plot(signal)
pass

In [ ]:
from scipy.signal import lfilter
f = lfilter(b, a, signal)
subplot(211)
plot(abs(rfft(f)), label='filtered')
xlim((0,80))
legend()
subplot(212)
plot(abs(rfft(signal)), label='original')
legend()
xlim((0,80))

In [ ]:
plot(angle(rfft(f)), label='filtered')
plot(angle(rfft(signal)), label='original')
xlabel('frequency')
ylabel('phase')
legend()
pass

In [ ]:
stem(angle(rfft(f)), linefmt='b:', label='filtered')
stem(angle(rfft(signal)), 'g.-.', label='original')

legend()
xlim(0,50)

In [ ]:
angle(r)[5], angle(r)[30]

In [ ]:
angle(rfft(f))[5] - angle(rfft(signal))[5]

In [ ]:
angle(rfft(f))[30] - angle(rfft(signal))[30]

hmmm....

In [ ]:
angle(rfft(f))[30] - angle(rfft(signal))[30] - (2 * pi)

In [ ]:
plot(signal)
plot(f)
xlim((0, 200))
grid()

## Excersise

1. Implement and test a [DC Blocker](https://ccrma.stanford.edu/~jos/fp/DC_Blocker.html) filter. That is, make a semi-complex signal with some DC offset. For instance, mix 3 sine waves of different frequencies with some constant, non-periodic signal with amplitude between -1 and 1. Then build a filter that, when applied to you signal, removes the DC but leaves the sine waves untouched. Show the frequency and phase response of your filter.

2. Audio signals commonly suffer from ["mains hum"](https://en.wikipedia.org/wiki/Mains_hum). Given this example of [mains hum](http://www.mat.ucsb.edu/201A/nb/Mains_hum_60_Hz_01.wav), design a filter to clean up [this noisy recording](http://www.mat.ucsb.edu/201A/nb/AlanWattsWithMainsHum.wav).


By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

Adapted by Karl Yerkes

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)